# problem statement:
What is the best font to use for my newsletter?




In [1]:
import requests
import pandas
from bs4 import BeautifulSoup
import re

In [2]:
# company_website_pairs = []
# with open('tech_websites.csv', newline='') as csvfile:
#     tech_websites = csv.reader(csvfile, dialect='excel')
#     for row in tech_websites:
#         # print(', '.join(row))
#         # name, website = row.split(',')
#         name, website = row[0], row[1]
#         company_website_pairs.append((name, website))
# # company_website_pairs

df = pandas.read_csv('tech_websites.csv', index_col=False)
df.head()

,company,website
0,youtube,https://youtube.com
1,google,https://google.com
2,gmail,https://gmail.com
3,google drive,https://drive.google.com
4,spotify,https://spotify.com


In [3]:
# TODO finalize output
# apple html
apple_df = df[df['company'] == 'apple']
URL = apple_df.iat[0,1] # :(


page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

font_tags = []
results = soup.find_all("link")
for result in results:
    # print(result.prettify())
    result_pretty = result.prettify()
    if "font" in result_pretty:
        font_tags.append(result_pretty)

# find 'fonts?families=' parse until "
sub = 'fonts?families='
families = "" 
for tag in font_tags:
    ind = tag.find(sub) + len(sub)

    for i in range(ind, len(tag)):
        if tag[i] == "\"":
            families = tag[ind:i]
            break
        else:
            i+=1
families


'SF+Pro,v3|SF+Pro+Icons,v3'

In [4]:
# youtube html
youtube_df = df[df['company'] == 'youtube']
URL = youtube_df.iat[0,1] # :(
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
# print(soup.prettify())
# html_tag = soup.find("html")
# print(html_tag)
style_string = soup.html['style']
print(style_string)

# extract font families
sub_string = "font-family:"
families = "" 
index = style_string.find(sub_string)
end_index = index + len(sub_string) -1
# print(style_string[index])
# print(style_string[end_index])
sub_string_end = ";"
fam_index = style_string.find(sub_string_end, end_index )
# print(fam_index)
# print(style_string[fam_index])
families = style_string[end_index+1: fam_index]
# print(families)
# x = re.search("^font-family: .*;$", style_string)
# print(x)
font_families = families.strip().split(',')
print(font_families)

font-size: 10px;font-family: Roboto, Arial, sans-serif;
['Roboto', ' Arial', ' sans-serif']


In [5]:
# Approach 1
bad = []
for ind, row in df.iterrows():
    # print(row)
    company = row[0]
    URL = row[1]
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        style_string = soup.html['style']
        # extract font families
        sub_string = "font-family:"
        families = "" 
        index = style_string.find(sub_string)
        if index == -1: raise 
        end_index = index + len(sub_string) -1
        sub_string_end = ";"
        fam_index = style_string.find(sub_string_end, end_index )
        if index == -1: raise 
        families = style_string[end_index+1: fam_index]
        font_families = families.strip().split(',')
        print("-----")
        print(company)
        print(font_families)
    except:
        bad.append(company)
print("------------------------")
print(bad)

-----
youtube
['Roboto', ' Arial', ' sans-serif']
------------------------
['google', 'gmail', 'google drive', 'spotify', 'twitter', 'microsoft', 'amazon', 'facebook', 'instagram', 'messenger', 'whatsapp', 'twitch', 'tiktok', 'apple', 'app store', 'zoom', 'cash app', 'netflix', 'discord', 'shopify', 'uber', 'paypal']


In [6]:
# # download all html
# for ind, row in df.iterrows():
#     company = row[0]
#     URL = row[1]
#     page = requests.get(URL)
#     soup = BeautifulSoup(page.content, "html.parser")
#     with open("{}.html".format(company), "w") as f:
#         f.write(soup.prettify())
#         print("-----")
#         print(company)

In [11]:
# find 'font' in html
no_font_websites = []
for ind, row in df.iterrows():
    company = row[0]
    URL = row[1]
    hasFont = False
    with open("{}.html".format(company), "r") as f:
        for line in f.readlines():
            # if line.find("family") != -1: 
            if line.find("font-family") != -1: 
                print(company)
                hasFont = True
                print(line)
        # print("-----")
        # print(company)
        # print("has font: " + str(hasFont))
        # if not hasFont:
        #     print(company)
        #     no_font_websites.append(company)

youtube
<html darker-dark-theme="" lang="en" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing="">

youtube
   #search-input.ytd-searchbox-spt input{-webkit-appearance:none;-webkit-font-smoothing:antialiased;background-color:transparent;border:none;box-shadow:none;color:inherit;font-family:Roboto,Noto,sans-serif;font-size:16px;font-weight:400;line-height:24px;margin-left:4px;max-width:100%;outline:none;text-align:inherit;width:100%;-ms-flex:1 1 0.000000001px;-webkit-flex:1;-webkit-box-flex:1;-moz-box-flex:1;flex:1;-webkit-flex-basis:0.000000001px;-ms-flex-preferred-size:0.000000001px;flex-basis:0.000000001px}#search-container.ytd-searchbox-spt{pointer-events:none;position:absolute;top:0;right:0;bottom:0;left:0}#search-input.ytd-searchbox-spt #search::-webkit-input-placeholder{color:#888}#search-input.ytd-searchbox-spt #search::-moz-input-placeholder{color:#888}#search-input.ytd-searchbox-spt #search:-ms-input-placeholder{co

In [8]:
# debugging specific html elements
target = "google"
youtube_df = df[df['company'] == target]
URL = youtube_df.iat[0,1] # :(
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())
html_tag = soup.find("html")
# print(html_tag)
with open("{}.html".format(target), "w") as f:
    f.write(soup.prettify())


<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
 <head>
  <meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for." name="description"/>
  <meta content="noodp" name="robots"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-law.gif" itemprop="image"/>
  <meta content="Seasonal Holidays 2022" property="twitter:title"/>
  <meta content="" property="twitter:description"/>
  <meta content="summary_large_image" property="twitter:card"/>
  <meta content="@GoogleDoodles" property="twitter:site"/>
  <meta content="https://www.google.com/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-2xa.gif" property="twitter:image"/>
  <meta content="https://www.google.com/logos/doodles/2022/seasonal-holidays-2022-6753651837109831.2-2xa.gif" pr

# Approach 2


In [36]:
no_font_websites = []
company_font_df = pandas.DataFrame()

for ind, row in df.iterrows():
    company = row[0]
    font_html = ""
    hasFont = False
    company_fonts = []

    with open("{}.html".format(company), "r") as f:
        for line in f.readlines():
            if line.find("font-family") != -1: 
                # print(company)
                hasFont = True
                # find fonts, they end in ; or }
                # find substrings between font-family: and ; or } --- regex
                if hasFont:
                    print(company)
                    # print(line)
                    regex1 = "(?<=font-family:)(.*?)(?=;|})"
                    matchObj = re.findall(regex1, line)
                    print(matchObj)

                else:
                    no_font_websites.append(company)


youtube
[' Roboto, Arial, sans-serif']
youtube
['Roboto,Noto,sans-serif']
google
['arial,sans-serif', 'inherit']
gmail
['arial,sans-serif']
gmail
["'Product Sans'"]
google drive
['arial,sans-serif']
google drive
["'Product Sans'"]
twitter
['-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Helvetica,Arial,sans-serif']
twitter
['inherit']
twitter
[' Helvetica, sans-serif']
instagram
['Placeholder Font', 'Placeholder Font', 'Optimistic Display Bold, system-ui, sans-serif', 'Optimistic Display Bold, system-ui, sans-serif', 'Optimistic Display Bold, system-ui, sans-serif', 'Placeholder Font', 'Placeholder Font', 'Optimistic Display Medium, system-ui, sans-serif', 'Placeholder Font', 'Placeholder Font', 'Placeholder Font', 'Optimistic Display Bold, system-ui, sans-serif', 'Optimistic Display Bold, system-ui, sans-serif', 'Optimistic Display Bold, system-ui, sans-serif', 'Placeholder Font', 'Placeholder Font', 'Optimistic Display Medium, system-ui, sans-serif', 'Placeholder Font']
paypal
[]

# Approach 3 - Chat GPT

In [1]:
import requests
from bs4 import BeautifulSoup

# specify the URL of the website you want to parse
url = "https://www.apple.com"

# send a request to the website and get the HTML content
response = requests.get(url)
html_content = response.text

# use BeautifulSoup to parse the HTML and extract the font used on the website
soup = BeautifulSoup(html_content, "html.parser")
font_element = soup.find("span", {"class": "font-family"})

# print the font used on the website
if font_element:
    print(font_element.text)
else:
    print("Could not find font used on website.")


Could not find font used on website.


In [3]:
import requests
from bs4 import BeautifulSoup

# specify the URL of the website you want to parse
url = "https://www.apple.com"

# send a request to the website and get the HTML content
response = requests.get(url)
html_content = response.text

# use BeautifulSoup to parse the HTML and extract the font used on the website
soup = BeautifulSoup(html_content, "html.parser")
style_elements = soup.find_all("style")

# search the style elements for the font used on the website
font_used = ""
for style_element in style_elements:
    if "font-family" in style_element.text:
        font_used = style_element.text.split("font-family: ")[1].split(";")[0]
        break

# print the font used on the website
if font_used:
    print(font_used)
else:
    print("Could not find font used on website.")


ConnectionError: HTTPSConnectionPool(host='www.apple.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10dc7da50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))